# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera



## Table of contents

* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction: Business Problem <a name="introduction"></a>


In this project we will try to find an all places that have been categorise as **'Halal' place** in the map. It is to help the muslim Traveler to see the where the place they can go and also for the Muslim Travel agency to plan up a good package for their Business.

Islam people or muslim is one of the religion that have a strict selection of **food and places** due to their believe and reliegion matter. However, the number of muslim traveler is rising up in this few dacade, so we decided to analyze the halal places that may the muslim traveler and business  traveler agency want to visit.

Although there are many website that suggesting places to visit for muslim, but there is a question problem that how much places in New York are actually listed as **Halal in the map**.


## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing halal places in New York
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:

* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Berlin center will be obtained using **Google Maps API geocoding** of well known Malaysia location (Kuala Lumpur)

* ### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest. 

Let's first find the latitude & longitude of New York as the center, using specific, well known address and Google Maps geocoding API.

In [6]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [7]:
CLIENT_ID = 'LBRMX4ESDDDX5KIXUL050GFTIZKHFZFQDYTBQN0ER4S50JPY' # your Foursquare ID
CLIENT_SECRET = 'PCMUFO5VM0YGIWCDW13KGD2TXCTIXWS35GSBB4PCGDMG1XRP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LBRMX4ESDDDX5KIXUL050GFTIZKHFZFQDYTBQN0ER4S50JPY
CLIENT_SECRET:PCMUFO5VM0YGIWCDW13KGD2TXCTIXWS35GSBB4PCGDMG1XRP


In [8]:
address = 'New York City'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [13]:
search_query = 'Halal '
radius = 10000
print(search_query + ' .... OK!')

Halal  .... OK!


In [14]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LBRMX4ESDDDX5KIXUL050GFTIZKHFZFQDYTBQN0ER4S50JPY&client_secret=PCMUFO5VM0YGIWCDW13KGD2TXCTIXWS35GSBB4PCGDMG1XRP&ll=40.7127281,-74.0060152&v=20180604&query=Halal &radius=10000&limit=30'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1f7d3a1952365470a6c075'},
 'response': {'venues': [{'id': '4c1dd42bb306c928f83566b7',
    'name': 'Halal Chicken & Gyro',
    'location': {'address': '55th',
     'lat': 40.71344,
     'lng': -74.008173,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71344,
       'lng': -74.008173}],
     'distance': 198,
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['55th', 'New York, NY', 'United States']},
    'categories': [],
    'referralId': 'v-1595899229',
    'hasPerk': False},
   {'id': '4c87bbde33453704e8361572',
    'name': 'Zaika Halal Street Cart @ Duane St',
    'location': {'address': 'Broadway & Duane St',
     'lat': 40.714975,
     'lng': -74.005877,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.714975,
       'lng': -74.005877}],
     'distance': 250,
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United Sta

In [16]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

C:\Users\hafiz\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.crossStreet,location.postalCode,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4c1dd42bb306c928f83566b7,Halal Chicken & Gyro,[],v-1595899229,False,55th,40.713440,-74.008173,"[{'label': 'display', 'lat': 40.71344, 'lng': ...",198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4c87bbde33453704e8361572,Zaika Halal Street Cart @ Duane St,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1595899229,False,Broadway & Duane St,40.714975,-74.005877,"[{'label': 'display', 'lat': 40.714975, 'lng':...",250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5604422e498e4359dc2997ee,Egyptian Guys Halal Food,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1595899229,False,Grand St,40.715730,-74.006891,"[{'label': 'display', 'lat': 40.71572970708845...",342,...,Clinton St,10002,Lower East Side,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4cb89e85035d236aff92ce4e,Halal Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1595899229,False,503 Broadway,40.715330,-74.011395,"[{'label': 'display', 'lat': 40.71532996030887...",538,...,Broome & Spring,10012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4d90b24e788c548188176afd,The Halal Guy,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1595899229,False,118 Water St,40.705455,-74.007498,"[{'label': 'display', 'lat': 40.705455, 'lng':...",819,...,wall st and john st,10005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Halal Chicken & Gyro,None,55th,40.713440,-74.008173,"[{'label': 'display', 'lat': 40.71344, 'lng': ...",198,US,New York,NY,United States,"[55th, New York, NY, United States]",NaN,NaN,NaN,4c1dd42bb306c928f83566b7
1,Zaika Halal Street Cart @ Duane St,Food Truck,Broadway & Duane St,40.714975,-74.005877,"[{'label': 'display', 'lat': 40.714975, 'lng':...",250,US,New York,NY,United States,"[Broadway & Duane St, New York, NY, United Sta...",NaN,NaN,NaN,4c87bbde33453704e8361572
2,Egyptian Guys Halal Food,Food Truck,Grand St,40.715730,-74.006891,"[{'label': 'display', 'lat': 40.71572970708845...",342,US,New York,NY,United States,"[Grand St (Clinton St), New York, NY 10002, Un...",Clinton St,10002,Lower East Side,5604422e498e4359dc2997ee
3,Halal Truck,Food Truck,503 Broadway,40.715330,-74.011395,"[{'label': 'display', 'lat': 40.71532996030887...",538,US,New York,NY,United States,"[503 Broadway (Broome & Spring), New York, NY ...",Broome & Spring,10012,NaN,4cb89e85035d236aff92ce4e
4,The Halal Guy,Food Truck,118 Water St,40.705455,-74.007498,"[{'label': 'display', 'lat': 40.705455, 'lng':...",819,US,New York,NY,United States,"[118 Water St (wall st and john st), New York,...",wall st and john st,10005,NaN,4d90b24e788c548188176afd
5,Halal Food Truck,Food Truck,Wall Street & Water Street,40.705002,-74.008021,"[{'label': 'display', 'lat': 40.70500214280706...",876,US,New York,NY,United States,"[Wall Street & Water Street, New York, NY 1000...",NaN,10005,NaN,4cea0c3ef8653704eebebac4
6,Halal Food Cart,Indian Restaurant,723,40.712778,-74.005943,"[{'label': 'display', 'lat': 40.71277753095443...",8,US,New York,NY,United States,"[723 (Btwn w 23rd & 24th Street), New York, NY...",Btwn w 23rd & 24th Street,10010,NaN,4c1102f3ce57c928ecbb82d2
7,The Halal Guys,Halal Restaurant,307 E 14th St,40.732304,-73.984401,"[{'label': 'display', 'lat': 40.73230359462967...",2841,US,New York,NY,United States,"[307 E 14th St (btwn 1st & 2nd Ave), New York,...",btwn 1st & 2nd Ave,10003,Midtown,523cc0df11d2ba091e493432
8,Adel's best #1 Halal Food Cart,Falafel Restaurant,Maiden Lane & Front street,40.705609,-74.005599,"[{'label': 'display', 'lat': 40.70560923100874...",793,US,New York,NY,United States,[Maiden Lane & Front street (Maiden Lane & Fro...,Maiden Lane & Front street,10038,NaN,4bba18d11261d13adc36ea98
9,Continental Express Halal,Food Truck,Whitehall St.,40.704950,-74.013071,"[{'label': 'display', 'lat': 40.70494974273545...",1050,US,New York,NY,United States,"[Whitehall St. (State St.), New York, NY, Unit...",State St.,NaN,NaN,4dfa355b52b1a82364e5de01


In [18]:
dataframe_filtered.name

0                     Halal Chicken & Gyro
1       Zaika Halal Street Cart @ Duane St
2                 Egyptian Guys Halal Food
3                              Halal Truck
4                            The Halal Guy
5                         Halal Food Truck
6                          Halal Food Cart
7                           The Halal Guys
8           Adel's best #1 Halal Food Cart
9                Continental Express Halal
10    Halal Food Cart (Liberty Halal Food)
11                Halal Food Cart @ UniQlo
12                        Asian Halal Food
13                         Halal Food Cart
14                       #1 Soho Halal Guy
15                      Sammy's Halal Food
16               Hasan & Abdul Halal Truck
17                        Halal food truck
18                             Halal Truck
19                Sarwari Halal Food Truck
20                         Halal Food Cart
21                          The Halal Guys
22                     Rafiqi's Halal Food
23         

In [19]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting list of Halal Places in New York. Keyword 'Halal' is used to define the target places.

In first step we have collected the required  (New York City). We have also **identified halal places** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**Halal**' across different areas of New York - Database wwere define and being pun into the table for listing up the places

In third and final step we will **markers** to point out the places in the map araound the New York City Area.

## Analysis <a name="analysis"></a>

Lets count the precentage or places halal restaurant in New York City

In [55]:
search_query != 'halal '
radius = 10000
print(search_query + ' .... OK!')

restaurant  .... OK!


In [50]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1f7fe751d07f34bc730207'},
 'response': {'venues': [{'id': '4c1dd42bb306c928f83566b7',
    'name': 'Halal Chicken & Gyro',
    'location': {'address': '55th',
     'lat': 40.71344,
     'lng': -74.008173,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71344,
       'lng': -74.008173}],
     'distance': 198,
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['55th', 'New York, NY', 'United States']},
    'categories': [],
    'referralId': 'v-1595899918',
    'hasPerk': False},
   {'id': '4c87bbde33453704e8361572',
    'name': 'Zaika Halal Street Cart @ Duane St',
    'location': {'address': 'Broadway & Duane St',
     'lat': 40.714975,
     'lng': -74.005877,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.714975,
       'lng': -74.005877}],
     'distance': 250,
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United Sta

List up the Halal Restaurant

In [51]:

dataframe = json_normalize(venues)
dataframe.head()

C:\Users\hafiz\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.crossStreet,location.postalCode,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4c1dd42bb306c928f83566b7,Halal Chicken & Gyro,[],v-1595899830,False,55th,40.713440,-74.008173,"[{'label': 'display', 'lat': 40.71344, 'lng': ...",198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4c87bbde33453704e8361572,Zaika Halal Street Cart @ Duane St,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1595899830,False,Broadway & Duane St,40.714975,-74.005877,"[{'label': 'display', 'lat': 40.714975, 'lng':...",250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5604422e498e4359dc2997ee,Egyptian Guys Halal Food,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1595899830,False,Grand St,40.715730,-74.006891,"[{'label': 'display', 'lat': 40.71572970708845...",342,...,Clinton St,10002,Lower East Side,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4cb89e85035d236aff92ce4e,Halal Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1595899830,False,503 Broadway,40.715330,-74.011395,"[{'label': 'display', 'lat': 40.71532996030887...",538,...,Broome & Spring,10012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4d90b24e788c548188176afd,The Halal Guy,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1595899830,False,118 Water St,40.705455,-74.007498,"[{'label': 'display', 'lat': 40.705455, 'lng':...",819,...,wall st and john st,10005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a some number of halal places in New york (only 5 from 30 listed halal places are categories as restaurant), there are low restaurant density fairly close to city center. Highest concentration of Halal places was detected near by the Brooklyn Bridge City Hall, , but it might be hard for Business agent traveler to make a pakage which offer a combination of popularity among tourists, closeness to city center, strong socio-economic dynamics *and* a number of pockets of low restaurant density.

Those location candidates were then clustered to create zones of interest which contain greatest number of halal place at New York. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 15 zones containing largest number of Halal Place locations based on the existing venues - both . This, of course, does not imply that those zones are actually not really optimal locations for a muslim traveler (base on foursquare map) Purpose of this analysis was to only provide info on areas close to New York center. It is entirely possible that there is a very good reason for some number of halal restaurants in any of those areas, reasons which would make them unsuitable for a new halal restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>